In [30]:
import pandas as pd
import re
import csv
import requests


In [31]:
def clean_text_df(df):
    # Create a copy to avoid modifying the original
    df = df.copy()
    
    # Dictionary for character replacements
    replacements = {
        'á': 'a',  'Á': 'a',
        'é': 'e', 'É': 'e', 
        'í': 'i',  'Í': 'i', 
        'ó': 'o',   'Ó': 'o',
        'ú': 'u', 'Ú': 'u',
        'ñ': 'n', 'Ñ': 'n'
    }
    
    # Function to apply replacements
    def replace_chars(text):
        if isinstance(text, str):
            for old, new in replacements.items():
                text = text.replace(old, new)
            return text.lower()
        return text
    
    # Clean column names
    df.columns = [replace_chars(col) for col in df.columns]
    
    # Clean string values
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].apply(replace_chars)
    
    return df

### Get the products
get all the purchases, corresponding to the 3 FAs. 

In [32]:
# Read the CSV file
df17 = pd.read_csv("../raw_data/ConvenioMarco/vehiculos_2017/MaestraProd_cm_2239-4-lr17.csv", sep=';', encoding='cp1252')  
df21 = pd.read_csv("../raw_data/ConvenioMarco/vehiculos_2021/MaestraProd_cm_2239-5-lr21.csv", sep=';',  encoding='cp1252')
df23 = pd.read_csv('../raw_data/ConvenioMarco/vehiculos_2023/MaestraProd_cm_2239-8-lr23.csv', sep=';', encoding='utf-8-sig', encoding_errors='replace')

df17 = clean_text_df(df17)
df21 = clean_text_df(df21)
df23 = clean_text_df(df23)


In [33]:
print(df17.columns, df21.columns, df23.columns)
#change var of 17 and 21 to match 23
rename_dict = {'nrolicitacionpublica': 'numero licitacion',
               'idproducto': 'id producto',
               'idproductocm': 'id producto', 
               'rut': 'rut proveedor',
               'idproveedor': 'id proveedor',
                }
df17.rename(columns=rename_dict, inplace=True)
df21.rename(columns=rename_dict, inplace=True)



# Print columns that differ between dataframes
cols_diff = (set(df17.columns) ^ set(df21.columns) ^ set(df23.columns))
print("Different columns:", cols_diff if cols_diff else "None - all columns match")

df17['year'] = 2017
df21['year'] = 2021
df23['year'] = 2023
df = pd.concat([df17, df21, df23], axis=0, ignore_index=True)


Index(['id convenio marco', 'nrolicitacionpublica', 'idproducto',
       'nombre producto', 'idtipoproducto', 'tipo producto', 'marca', 'modelo',
       'medida', 'descripcion', 'idproveedor', 'rut', 'proveedor', 'precio',
       'moneda', 'estado productoproveedorconvenio',
       'estado proveedor convenio', 'estado producto', 'fechaactualizacion'],
      dtype='object') Index(['nrolicitacionpublica', 'idproductocm', 'nombreproducto',
       'idtipoproducto', 'tipoproducto', 'marca', 'modelo', 'medida',
       'descripcion', 'rut', 'orgcode', 'razonsocialproveedor', 'moneda',
       'precio', 'tipo_precio', 'stock', 'fechaactualizacion'],
      dtype='object') Index(['convenio marco', 'numero licitacion', 'id convenio marco',
       'id proveedor', 'nombre proveedor', 'rut proveedor', 'id producto',
       'codigo onu', 'producto', 'id tipo producto', 'tipo producto', 'region',
       'marca', 'modelo', 'medida', 'stock', 'precio en tienda',
       'precio regular', 'precio oferta', 

In [34]:
# Get unique values from the "tipo producto" column
unique_tipos = df['tipo producto'].value_counts()
print(unique_tipos)

print(df['tipo producto'].unique())


tipo producto
suv                                                 5154
camioneta                                           4052
cargo                                               2752
sedan                                               1114
tolva                                                633
excavadora                                           506
furgon                                               485
minibus                                              393
cargador                                             358
aditamento para maquinaria                           298
mantencion preventiva vehiculo liviano y mediano     271
mantencion preventiva vehiculo pesado                250
hatchback                                            144
bus pesado                                           133
retroexcavadora                                      112
tractor                                               87
buses media y larga distancia                         81
tracto           

### select relevant categories and obtain their brand and model

In [35]:
# List of Tipo Producto we're interested in: needs to have more than 100 obs and be reasonable small (no trucks, buses, tolva, etc)
tipos_producto = ['camioneta', 'sedan', 'suv', 'minibus', 'furgon', 'hatchback']

# Create an empty list to store all results
all_results = []
all_results_v2 = []
# Iterate through each Tipo Producto
for tipo in tipos_producto:
    print(f"\nTipo Producto: {tipo}")
    
    # Filter the dataframe for the current Tipo Producto
    df_filtered = df[df['tipo producto'] == tipo]
    
    # Get unique combinations of Marca and Modelo
    unique_combinations = df_filtered[['marca', 'modelo']].drop_duplicates()
    unique_combinations_v2 = df_filtered[['marca', 'modelo', 'numero licitacion', 'id producto']].drop_duplicates()

    
    # Sort the combinations by marca and Modelo
    unique_combinations = unique_combinations.sort_values(['marca', 'modelo'])
    unique_combinations_v2 = unique_combinations_v2.sort_values(['marca', 'modelo', 'numero licitacion', 'id producto'])

    # Add Tipo Producto column
    unique_combinations['tipo producto'] = tipo
    unique_combinations_v2['tipo producto'] = tipo

    # Append to all_results
    all_results.append(unique_combinations)
    all_results_v2.append(unique_combinations_v2)

# Combine all results into a single DataFrame
result_df = pd.concat(all_results, ignore_index=True)
result_df_v2 = pd.concat(all_results_v2, ignore_index=True)

# Save the result to a CSV file
output_file = 'csvs/unique_marca_modelo_by_tipo_producto.csv'
output_file_v2 = 'csvs/unique_marca_modelo_by_tipo_producto_ids.csv'

result_df.to_csv(output_file, index=False)
print(f"\nResults saved to {output_file}")

result_df_v2.to_csv(output_file_v2, index=False)
print(f"\nResults saved to {output_file_v2}")



Tipo Producto: camioneta

Tipo Producto: sedan

Tipo Producto: suv

Tipo Producto: minibus

Tipo Producto: furgon

Tipo Producto: hatchback

Results saved to csvs/unique_marca_modelo_by_tipo_producto.csv

Results saved to csvs/unique_marca_modelo_by_tipo_producto_ids.csv


### Vemos que en el nombre del producto hay varias especificaciones aparte del nombre, como el motor, drive, transmisión y año. Todos son del 2024.

In [ ]:
# Read the CSV data into a DataFrame
df = pd.read_csv('csvs/unique_marca_modelo_by_tipo_producto.csv')
df_v2 = pd.read_csv('csvs/unique_marca_modelo_by_tipo_producto_ids.csv')

def parse_modelo(modelo):
    parsed = {
        'Model Name': '',
        'Engine': '',
        'Drive': '',
        'Transmission': '',
        'Year': ''
    }
    tokens = modelo.strip().split()
    model_name_tokens = []
    started_fields = False

    for token in tokens:
        # Check for year (assuming it's a 4-digit number)
        if re.match(r'^\d{4}$', token):
            parsed['Year'] = token
            started_fields = True
        # Check for engine size with decimal or ending with 'L' or 'T'
        elif re.match(r'^\d+(\.\d+)?[LT]?T?$', token):
            if ('.' in token) or token.endswith(('L', 'T')):
                parsed['Engine'] = token
                started_fields = True
            else:
                if not started_fields:
                    model_name_tokens.append(token)
                else:
                    model_name_tokens.append(token)
        # Check for drive types
        elif token.upper() in ['4X2', '4X4', '2WD', '4WD', 'AWD', '2R', 'FWD', 'RWD']:
            parsed['Drive'] = token.upper()
            started_fields = True
        # Check for transmission types (including multi-character)
        elif re.match(r'^\d?(MT|AT|CVT|DCT|EAT6|EAT8)$', token.upper()):
            parsed['Transmission'] = token.upper()
            started_fields = True
        else:
            # Always add to Model Name
            model_name_tokens.append(token)

    parsed['Model Name'] = ' '.join(model_name_tokens).strip()
    return parsed


# Apply the parsing function to the DataFrame
parsed_data = df['modelo'].apply(parse_modelo)
parsed_data_v2 = df_v2['modelo'].apply(parse_modelo)

parsed_df = pd.DataFrame(parsed_data.tolist())
parsed_df_v2 = pd.DataFrame(parsed_data_v2.tolist())

# Combine the parsed data with the original DataFrame
result_df = pd.concat([df.drop(columns=['modelo']), parsed_df], axis=1)
result_df_v2 = pd.concat([df_v2.drop(columns=['modelo']), parsed_df_v2], axis=1)

# Save the result to a new CSV file
result_df.to_csv('csvs/parsed_output.csv', index=False)
result_df_v2.to_csv('csvs/parsed_output_ids.csv', index=False)
